# Profile Python

https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html summary:

* `%time`: Time the execution of a single statement
* `%timeit`: Time repeated execution of a single statement for more accuracy
* `%prun`: Run code with the profiler
* `%lprun`: Run code with the line-by-line profiler
* `%memit`: Measure the memory use of a single statement
* `%mprun`: Run code with the line-by-line memory profiler

See also <BR>
http://codedelish.com/resources/Python/profiling.html<BR>
http://pynash.org/2013/03/06/timing-and-profiling/

In [1]:
import sys
print(sys.version)

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]


In [2]:
!pip install line-profiler
!pip install psutil
!pip install memory_profiler

two functions that we will profile

In [3]:
def mult_by(inpt):
    """
    a simple function that is not complicated
    """
    avalu = inpt*2
    avalu+=1
    return

def primes(n=1000): 
    """
    A more complicated function to be profiled
    """
    A = [True] * (n+1)
    A[0] = False
    A[1] = False
    for i in range(2, int(n**0.5)):
        if A[i]:
            mult_by(i) # not actually relevant to determining a prime number
            for j in range(i**2, n+1, i):
                A[j] = False

    return [x for x in range(2, n) if A[x]]

# `time` magic function 

see https://ipython.readthedocs.io/en/stable/interactive/magics.html

How long does execution of a function take?

`%time` applies to a single line

In [18]:
%time primes(20) # call a function and provide an argument

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 26.9 µs


[2, 3, 5, 7, 11, 13, 17, 19]

for "microseconds" (us, 1E-6) versus "milliseconds" (ms, 1E-3), see https://en.wikipedia.org/wiki/Metric_prefix#List_of_SI_prefixes

similarly, `%%time` applies to the entire cell

In [19]:
%%time 
primes(20)
primes(30)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 37.4 µs


Displaying output skews measurement results for cell execution, so best practice is to store output to a variable

In [21]:
%time reslt=primes(20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 45.8 µs


There's not much difference for a small output, but the relevance increases when the input is higher

In [22]:
%time reslt=primes(1000000)

print('length of array:',len(reslt))

CPU times: user 480 ms, sys: 40 ms, total: 520 ms
Wall time: 635 ms
length of array: 78498


To run multiple timing test, use `%timeit`

In [23]:
%timeit reslt=primes(1000000)

483 ms ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Want to capture the output of `timeit` to a variable? See https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module

# cProfile

https://docs.python.org/3/library/profile.html

In [9]:
import cProfile

In [10]:
cProfile.run('primes(10000000)')

         451 function calls in 7.350 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      446    0.001    0.000    0.001    0.000 <ipython-input-3-9a7fc4f084d5>:1(mult_by)
        1    1.274    1.274    1.274    1.274 <ipython-input-3-9a7fc4f084d5>:22(<listcomp>)
        1    6.011    6.011    7.286    7.286 <ipython-input-3-9a7fc4f084d5>:9(primes)
        1    0.064    0.064    7.350    7.350 <string>:1(<module>)
        1    0.000    0.000    7.350    7.350 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




* `ncalls`:       How many times it was called
* `tottime`:      for the total time spent in the given function (and excluding time made in calls to sub-functions)
* `percall`:      is the quotient of tottime divided by ncalls
* `cumtime`:      is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
* `percall`:      is the quotient of cumtime divided by primitive calls
* `filename:lineno(function)`:    provides the respective data of each function

# line time

https://nyu-cds.github.io/python-performance-tuning/03-line_profiler/
    
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

In [11]:
%load_ext line_profiler

In [12]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

Profile the lines of the `prime` function

In [13]:
%lprun -f primes primes(10000)

Timer unit: 1e-06 s

Total time: 0.151453 s
File: <ipython-input-3-9a7fc4f084d5>
Function: primes at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def primes(n=1000): 
    10                                               """
    11                                               A more complicated function to be profiled
    12                                               """
    13         1         42.0     42.0      0.0      A = [True] * (n+1)
    14         1          5.0      5.0      0.0      A[0] = False
    15         1          2.0      2.0      0.0      A[1] = False
    16        99        179.0      1.8      0.1      for i in range(2, int(n**0.5)):
    17        98        173.0      1.8      0.1          if A[i]:
    18        25        353.0     14.1      0.2              mult_by(i) # not actually relevant to determining a prime number
    19     17006      68760.0      4.0     45.4              for j 

profile `mult_by` function when `primes()` is executed

In [14]:
%lprun -f mult_by primes(10000)

Timer unit: 1e-06 s

Total time: 0.00016 s
File: <ipython-input-3-9a7fc4f084d5>
Function: mult_by at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mult_by(inpt):
     2                                               """
     3                                               a simple function that is not complicated
     4                                               """
     5        25         91.0      3.6     56.9      avalu = inpt*2
     6        25         41.0      1.6     25.6      avalu+=1
     7        25         28.0      1.1     17.5      return

* `Line #`: The line number in the code
* `Hits`: The number of times that line was executed
* `Time`: The total amount of time spent executing the line in the timer’s units
* `Per Hit`: The average amount of time spent executing the line once in the timer’s unit
* `% Time`: The percentage of time spent on that line relative to the total amount of recorded time spent in the function
* `Line Contents`: The actual source code of the line

line_profiler generates profiles based on the lines in a program.

Line based profiling can provide more information about where a program is performing badly.